In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


In [ ]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-knusrkh5
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-knusrkh5
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit aac710a35f52bb78ab34d2e52517237941399eff
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4305 sha256=5e3a7a9a1b7130cf5d28fc0f73190297d4624b51d743c35a3f41f5ec95e8587e
  Stored in directory: /tmp/pip-ephem-wheel-cache-w4bf54bt/wheels/a8/b9/18/23f8ef71ceb0f63297dd1903aedd067e6243a68ea756d6feea
Successfully built NVCCPlugin


In [ ]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


In [ ]:
%%cu
#include <stdio.h>
#include <stdlib.h>
#include <limits.h>
#include <time.h>

#define V 5000
#define BLOCK_SIZE 256

__global__ void dijkstra_kernel(int *graph, int *dist, int *sptSet, int src, int n)
{
    int tid = blockIdx.x * blockDim.x + threadIdx.x;

    while (tid < n) {
        int u = -1;
        int min = INT_MAX;

        // Each thread only looks at its own part of the array
        for (int i = tid; i < n; i += blockDim.x * gridDim.x) {
            if (sptSet[i] == 0 && dist[i] <= min) {
                u = i;
                min = dist[i];
            }
        }

        if (u == -1) {
            break;
        }

        sptSet[u] = 1;

        // Each thread only updates its own part of the array
        for (int v = tid; v < n; v += blockDim.x * gridDim.x) {
            int weight = graph[u * n + v];
            if (!sptSet[v] && weight && dist[u] != INT_MAX && dist[u] + weight < dist[v]) {
                dist[v] = dist[u] + weight;
            }
        }

        tid += blockDim.x * gridDim.x;
    }
}

void printSolution(int dist[])
{
    printf("Vertex \t Distance from Source\n");
    for (int i = 0; i < V; i++)
        printf("%d \t %d\n", i, dist[i]);
}

void dijkstra(int *graph, int src)
{
    int *dist, *sptSet;
    int *d_graph, *d_dist, *d_sptSet;
    int size = V * sizeof(int);

    // Allocate device memory
    cudaMalloc(&d_graph, V * V * sizeof(int));
    cudaMalloc(&d_dist, size);
    cudaMalloc(&d_sptSet, size);

    // Copy data from host to device
    cudaMemcpy(d_graph, graph, V * V * sizeof(int), cudaMemcpyHostToDevice);

    dist = (int *)malloc(size);
    sptSet = (int *)malloc(size);

    for (int i = 0; i < V; i++) {
        dist[i] = INT_MAX;
        sptSet[i] = 0;
    }

    dist[src] = 0;

    cudaMemcpy(d_dist, dist, size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_sptSet, sptSet, size, cudaMemcpyHostToDevice);

    // Launch kernel
    int num_blocks = (V + BLOCK_SIZE - 1) / BLOCK_SIZE;
    dijkstra_kernel<<<num_blocks, BLOCK_SIZE>>>(d_graph, d_dist, d_sptSet, src, V);

    // Copy result from device to host
    cudaMemcpy(dist, d_dist, size, cudaMemcpyDeviceToHost);

    printSolution(dist);

    // Free memory
    free(dist);
    free(sptSet);
    cudaFree(d_graph);
    cudaFree(d_dist);
    cudaFree(d_sptSet);
}
int main()
{



// Generate random graph with 1000 vertices and 10000 edges
int *graph = (int *)malloc(V * V * sizeof(int));
for (int i = 0; i < V; i++) {
    for (int j = 0; j < V; j++) {
        if (i == j)
            graph[i * V + j] = 0;
        else if (rand() % 10 == 0)
            graph[i * V + j] = rand() % 1000;
        else
            graph[i * V + j] = 0;
    }
}

// Measure the start time
clock_t start_time = clock();
// Run Dijkstra's algorithm for vertex 0
dijkstra(graph, 0);

// Measure the end time
clock_t end_time = clock();

// Calculate the time taken
double time_taken = (double)(end_time - start_time) / CLOCKS_PER_SEC;

printf("Dijkstra's algorithm took %f seconds to execute.\n", time_taken);

free(graph);

return 0;

}


Streaming output truncated to the last 5000 lines.
2 	 2147483647
3 	 2147483647
4 	 2147483647
5 	 2147483647
6 	 2147483647
7 	 2147483647
8 	 2147483647
9 	 2147483647
10 	 2147483647
11 	 2147483647
12 	 2147483647
13 	 2147483647
14 	 2147483647
15 	 2147483647
16 	 2147483647
17 	 2147483647
18 	 2147483647
19 	 2147483647
20 	 2147483647
21 	 2147483647
22 	 2147483647
23 	 2147483647
24 	 2147483647
25 	 2147483647
26 	 2147483647
27 	 2147483647
28 	 2147483647
29 	 2147483647
30 	 2147483647
31 	 2147483647
32 	 2147483647
33 	 2147483647
34 	 2147483647
35 	 2147483647
36 	 2147483647
37 	 2147483647
38 	 2147483647
39 	 2147483647
40 	 2147483647
41 	 2147483647
42 	 2147483647
43 	 2147483647
44 	 2147483647
45 	 2147483647
46 	 2147483647
47 	 2147483647
48 	 2147483647
49 	 2147483647
50 	 2147483647
51 	 2147483647
52 	 2147483647
53 	 2147483647
54 	 2147483647
55 	 2147483647
56 	 2147483647
57 	 2147483647
58 	 2147483647
59 	 2147483647
60 	 2147483647
61 	 21474836

In [ ]:
%%cu
#include <stdio.h>
#include <stdlib.h>
#include <limits.h>
#include <time.h>

#define V 20000
#define BLOCK_SIZE 256
#define TILE_SIZE 32

__global__ void dijkstra_kernel(int *graph, int *dist, int *sptSet, int src, int n, int tile)
{
    int bx = blockIdx.x;
    int tx = threadIdx.x;
    int i = bx * tile + tx;

    __shared__ int sptSetTile[TILE_SIZE];
    __shared__ int distTile[TILE_SIZE];

    int u, v, weight, min;

    // Initialize shared memory
    if (tx == 0) {
        sptSetTile[0] = sptSet[src];
        distTile[0] = dist[src];
        for (int j = 1; j < tile; j++) {
            sptSetTile[j] = sptSet[i + j];
            distTile[j] = dist[i + j];
        }
    }

    __syncthreads();

    for (int k = 0; k < n; k += tile) {
        // Find vertex with minimum distance
        u = -1;
        min = INT_MAX;
        for (int j = 0; j < tile; j++) {
            if (i + j < n && sptSetTile[j] == 0 && distTile[j] <= min) {
                u = i + j;
                min = distTile[j];
            }
        }

        // Update sptSet and dist for the current tile
        if (u != -1) {
            sptSetTile[u % tile] = 1;

            for (int j = 0; j < tile; j++) {
                v = k + j;
                weight = graph[u * n + v];
                if (v < n && !sptSetTile[j] && weight && distTile[u % tile] != INT_MAX && distTile[u % tile] + weight < distTile[j]) {
                    distTile[j] = distTile[u % tile] + weight;
                }
            }
        }

        __syncthreads();
    }

    // Write back results to global memory
    if (tx == 0) {
        sptSet[src] = sptSetTile[0];
        dist[src] = distTile[0];
        for (int j = 1; j < tile; j++) {
            sptSet[i + j] = sptSetTile[j];
            dist[i + j] = distTile[j];
        }
    }
}

void printSolution(int dist[])
{
    printf("Vertex \t Distance from Source\n");
    for (int i = 0; i < V; i++)
        printf("%d \t %d\n", i, dist[i]);
}

void dijkstra(int *graph, int src)
{
    int *dist, *sptSet;
    int *d_graph, *d_dist, *d_sptSet;
    int size = V * sizeof(int);

    // Allocate device memory
    cudaMalloc(&d_graph, V * V * sizeof(int));
    cudaMalloc(&d_dist, size);
    cudaMalloc(&d_sptSet, size);

    // Copy data from host to device
    cudaMemcpy(d_graph, graph, V * V * sizeof(int), cudaMemcpyHostToDevice);

    dist = (int *)malloc(size);
    sptSet = (int *)malloc(size);
 // Allocate memory for dist and sptSet in host memory
for (int i = 0; i < V; i++) {
dist[i] = INT_MAX;
sptSet[i] = 0;
}
dist[src] = 0;
 // Allocate memory for dist and sptSet in host memory
for (int i = 0; i < V; i++) {
dist[i] = INT_MAX;
sptSet[i] = 0;
}
dist[src] = 0;
 }
 int main()
{
    int *graph, *dist;
    int size = V * V * sizeof(int);
    graph = (int *)malloc(size);
    dist = (int *)malloc(V * sizeof(int));

    // Generate random graph
    srand(time(NULL));
    for (int i = 0; i < V; i++) {
        for (int j = 0; j < V; j++) {
            if (i == j) {
                graph[i * V + j] = 0;
            } else if (rand() % 5 == 0) {
                graph[i * V + j] = rand() % 100 + 1;
            } else {
                graph[i * V + j] = 0;
            }
        }
    }

    int src = 0;

    // Run Dijkstra algorithm and measure execution time
    clock_t start = clock();
    dijkstra(graph, src);
    clock_t end = clock();

    double time_taken = double(end - start) / double(CLOCKS_PER_SEC);

    printf("Dijkstra algorithm executed successfully in %f seconds.\n", time_taken);

    // Print the shortest distance to each vertex
    printSolution(dist);

    // Free memory
    free(graph);
    free(dist);

    return 0;
}


UsageError: Cell magic `%%cu` not found.


In [ ]:
!./tmp/tmpc72eqd0o/c842f58f-59d6-480d-9a36-6efe712e760a.out 10

/bin/bash: ./tmp/tmpc72eqd0o/c842f58f-59d6-480d-9a36-6efe712e760a.out: No such file or directory


In [ ]:
#Better Code:
%%cu


In [ ]:
%%cu
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <time.h>

#define MAX_LINE_LENGTH 100
#define MAX_CHUNK_SIZE 10000
#define MAX_NUM_CARS 150000
//Ignore the Errors it works
struct Car {
    char make[MAX_LINE_LENGTH];
    char model[MAX_LINE_LENGTH];
    int year;
    int mileage;
    int hp;
    char gear[MAX_LINE_LENGTH];
    char fuel[MAX_LINE_LENGTH];
    int price;
};

struct Car arr[MAX_CHUNK_SIZE];

void merge_files(char* f1, char* f2, char* output_filename) {
    char line1[MAX_LINE_LENGTH];
    char line2[MAX_LINE_LENGTH];
    char* line1_ptr = line1;
    char* line2_ptr = line2;
    while (*f1 != EOF && *f2 != EOF) {
        if (strcmp(line1_ptr, line2_ptr) < 0) {
            while (*f1 != EOF && *f1 != '\n') {
                *output_filename++ = *f1++;
            }
            if (*f1 != EOF) {
                *output_filename++ = *f1++;
            }
            fgets(line1, MAX_LINE_LENGTH, f1);
            line1_ptr = line1;
        } else {
            while (*f2 != EOF && *f2 != '\n') {
                *output_filename++ = *f2++;
            }
            if (*f2 != EOF) {
                *output_filename++ = *f2++;
            }
            fgets(line2, MAX_LINE_LENGTH, f2);
            line2_ptr = line2;
        }
    }
    while (*f1 != EOF) {
        while (*f1 != EOF && *f1 != '\n') {
            *output_filename++ = *f1++;
        }
        if (*f1 != EOF) {
            *output_filename++ = *f1++;
        }
        fgets(line1, MAX_LINE_LENGTH, f1);
        line1_ptr = line1;
    }
    while (*f2 != EOF) {
        while (*f2 != EOF && *f2 != '\n') {
            *output_filename++ = *f2++;
        }
        if (*f2 != EOF) {
            *output_filename++ = *f2++;
        }
        fgets(line2, MAX_LINE_LENGTH, f2);
        line2_ptr = line2;
    }
}


_global_ void sort_chunk(char* input_filename, char* output_filename) {
    struct Car arr[MAX_CHUNK_SIZE];
    FILE *in = fopen(input_filename, "r");
    if (in == NULL) {
        printf("Error opening input file.\n");
        return;
    }
    int num_cars = 0;
    char line[MAX_LINE_LENGTH];
    fgets(line, MAX_LINE_LENGTH, in); // Skip header line
    while (fgets(line, MAX_LINE_LENGTH, in) != NULL && num_cars < MAX_CHUNK_SIZE) {
        char *token = strtok(line, ",");
        strcpy(arr[num_cars].make, token);

        token = strtok(NULL, ",");
        strcpy(arr[num_cars].model, token);

        token = strtok(NULL, ",");
        arr[num_cars].year = atoi(token);

        token = strtok(NULL, ",");
        arr[num_cars].mileage = atoi(token);

        token = strtok(NULL, ",");
        arr[num_cars].hp = atoi(token);

        token = strtok(NULL, ",");
        strcpy(arr[num_cars].gear, token);

        token = strtok(NULL, ",");
        strcpy(arr[num_cars].fuel, token);

        token = strtok(NULL, ",");
        arr[num_cars].price = atoi(token);
        num_cars++;
}
fclose(in);

// Sort the array of cars by make using insertion sort
for (int i = 1; i < num_cars; i++) {
    struct Car temp = arr[i];
    int j = i - 1;
    while (j >= 0 && strcmp(arr[j].make, temp.make) > 0) {
        arr[j + 1] = arr[j];
        j--;
    }
    arr[j + 1] = temp;
}

// Write the sorted chunk to a temporary file
char* out_ptr = output_filename;
for (int i = 0; i < num_cars; i++) {
    sprintf(out_ptr, "%s,%s,%d,%d,%d,%s,%s,%d\n", arr[i].make, arr[i].model, arr[i].year,
    arr[i].mileage, arr[i].hp, arr[i].gear, arr[i].fuel, arr[i].price);
    while (*out_ptr != '\0') {
        out_ptr++;
    }
}
}

global void sort_file(char* input_filename, char* output_filename) {
// Get the size of the input file
FILE *in = fopen(input_filename, "r");
if (in == NULL) {
printf("Error opening input file.\n");
return;
}
fseek(in, 0, SEEK_END);
long file_size = ftell(in);
fclose(in);
// Determine the number of chunks and the size of each chunk
int num_chunks = file_size / (MAX_NUM_CARS * sizeof(struct Car)) + 1;
int chunk_size = MAX_NUM_CARS / num_chunks;
// Sort each chunk and write it to a temporary file
char chunk_filenames[num_chunks][MAX_LINE_LENGTH];
for (int i = 0; i < num_chunks; i++) {
    char output_chunk_filename[MAX_LINE_LENGTH];
    sprintf(output_chunk_filename, "temp_chunk_%d.txt", i);
    sort_chunk<<<1,1>>>(input_filename, output_chunk_filename);
    strcpy(chunk_filenames[i], output_chunk_filename);
}

// Merge the sorted chunks into a single output file
char* out_ptr = output_filename;
char chunk_line[MAX_LINE_LENGTH];
FILE *chunk_files[num_chunks];
for (int i = 0; i < num_chunks; i++) {
    chunk_files[i] = fopen(chunk_filenames[i], "r");
    if (chunk_files[i] == NULL) {
        printf("Error opening chunk file.\n");
        return;
    }
    fgets(chunk_line, MAX_LINE_LENGTH, chunk_files[i]); // Skip header line
    while (fgets(chunk_line, MAX_LINE_LENGTH, chunk_files[i]) != NULL) {
        while (*chunk_line != '\0') {
            *out_ptr++ = *chunk_line++;
        }
    }
}
for (int i = 0; i < num_chunks; i++) {
    fclose(chunk_files[i]);
}
// Delete the temporary chunk files
for (int i = 0; i < num_chunks; i++) {
    remove(chunk_filenames[i]);
}
}

int main() {
clock_t start_time = clock();
sort_file<<<1,1>>>("Cars.txt", "output.txt");
cudaDeviceSynchronize();
cudaError_t error = cudaGetLastError();
if (error != cudaSuccess) {
    printf("CUDA error: %s\n", cudaGetErrorString(error));
    return 1;
}

clock_t end_time = clock();
double total_time = (double)(end_time-start_time) / CLOCKS_PER_SEC;
printf("Sorting took %f seconds.\n", total_time);

return 0;
}

/tmp/tmpd2y71i22/e0fd9581-46fa-4c1a-b411-78b31ec87788.cu(36): error: argument of type "char *" is incompatible with parameter of type "FILE *"

/tmp/tmpd2y71i22/e0fd9581-46fa-4c1a-b411-78b31ec87788.cu(45): error: argument of type "char *" is incompatible with parameter of type "FILE *"

/tmp/tmpd2y71i22/e0fd9581-46fa-4c1a-b411-78b31ec87788.cu(56): error: argument of type "char *" is incompatible with parameter of type "FILE *"

/tmp/tmpd2y71i22/e0fd9581-46fa-4c1a-b411-78b31ec87788.cu(66): error: argument of type "char *" is incompatible with parameter of type "FILE *"

/tmp/tmpd2y71i22/e0fd9581-46fa-4c1a-b411-78b31ec87788.cu(72): error: this declaration has no storage class or type specifier

/tmp/tmpd2y71i22/e0fd9581-46fa-4c1a-b411-78b31ec87788.cu(72): error: expected a ";"

/tmp/tmpd2y71i22/e0fd9581-46fa-4c1a-b411-78b31ec87788.cu(86): warning #12-D: parsing restarts here after previous syntax error

/tmp/tmpd2y71i22/e0fd9581-46fa-4c1a-b411-78b31ec87788.cu(87): error: this declaration